# Índices Educación Superior Periodo 2005-2023

## I. Introducción
La educación es uno de los pilares de la sociedad, por tanto se hace interesante comprender cómo se relacionan algunos de sus indicadores y de que forma influyen unos con otros ¿existirá una relación entre el puntaje de corte de una carrera y su costo en arancel? ¿podemos agrupar carreras y universidades según su información de matrícula?

Se posee una base de datos histórica de educación superior que cuenta con registros desde 2005 a 2023 y que exploraremos a continuación.

## II. Exploración de datos

### Distribuciones importantes

En primer lugar, se decide contabilizar el porcentaje de registros correspondientes a carreras de pregrado y postgrado, resultando en un 83.8% y 16.2% respectivamente.

![Distribución de registros con respecto a la columna Pregrado/Posgrado](./outputs/dist.png)


Por otra parte, la distribución de estudiantes por regiones es la siguiente:

![Distribución de estudiantes por regiones](./outputs/count_students.png)

Aunque pudiese ser predecible, se evidencia una alta cantidad de estudiantes en la Región Metropolitana, seguida por Valparaíso y Biobío. 

En cuanto a la cantidad de estudiantes por tipo de carrera, se observa que la mayoría de los estudiantes se encuentran en carreras de tipo técnicas, seguidas por profesional con licenciatura y carreras profesionales.

![Distribución de estudiantes por tipo de carrera](./outputs/count_students_degree.png)

Agrupando las carreras por sector de conocimiento, se puede observar una gran presencia de estudiantes en carreras tecnológicas, seguidas por el área de administración y comercio. En tercer puesto, encontramos el área de salud que es casi equiparada por el área de educación.

![Distribución de estudiantes por sector de conocimiento](./outputs/count_students_area.png)

Por el lado de las estadísticas relacionadas al ingreso a estas carreras tenemos el promedio del puntaje obtenido en las pruebas de matemáticas y lenguaje. 
Los resultados registrados entre los años 2005 y 2019 corresponden a la prueba de selección universitaria PSU, con un puntaje mínimo de 150 puntos y un máximo de 850 puntos. Los siguientes 3 años corresponden a resultados de la prueba de transición universitaria PTU, mientras que el último año registrado corresponde a la nueva prueba de acceso a la educación superior PAES, cuyo rango de puntajes va desde los 100 hasta los 1.000 puntos.

A continuación se observa la distribución de los puntajes, donde se puede apreciar que en el caso de la PSU la mayoría de los estudiantes obtienen puntajes entre 400 y 700 puntos en ambas pruebas (esto ya que la escala de cada una de las pruebas de la PSU estaba estandarizado, con una media de 500 puntos y una desviación estándar de 110).


![Distribución de puntajes PSU](./outputs/psu.png)

![Distribución de puntajes PSU](./outputs/paes.png)

### Análisis de las columnas

Desde un punto de vista más detallado, se analizarán las columnas de la base de datos para determinar cuáles son las más relevantes para el análisis y cuáles podrían ser descartadas.

* Año: Se observa que la base de datos cuenta con registros desde el año 2005 hasta el año 2023. El tipo de dato de esta columna es de tipo entero, por lo que se necesitará convertir a tipo fecha para poder realizar análisis de series de tiempo.
* Cód. Institución: Corresponde al código que identifica a la institución de educación superior. El tipo de dato de esta columna es de tipo entero, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agrupaciones de datos.
* Nombre Institución: Corresponde al nombre de la institución de educación superior. El tipo de dato de esta columna es de tipo string, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agrupaciones de datos.
* Tipo Institución: Corresponde al tipo de institución de educación superior. El tipo de dato de esta columna es de tipo string, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agrupaciones de datos.
* Clasificación1 a Clasificación6: Corresponden a la clasificación de la institución de educación superior según factores como tipo de gestión (pública o privada), si pertenece al CRUCH, si se encuentra acreditada, entre otros. El tipo de dato de esta columna es de tipo string, no requiriendo ningún tipo de transformación. Podría ser útil para realizar agrupaciones de datos.

A partir de esto, notamos que hay columnas que no nos entregan información relevante o que podrían interferir con la generación de clusters a futuro, como el orden geografico de la región, el Código SIES o el idgenerocarrera (información demasiado específica). 

También la información referente a puntajes mínimos y máximos de ingreso es insuficiente (tienen una alta tasa de nulos), por lo que solo mantendremos la columna 'Promedio Puntaje (promedio matemáticas y lenguaje)' de las pruebas de ingreso. Las menciones o especialidades no están presentes en todas las carreras y la columna con el nombre del campus tiene una tasa de nulos del 57%, por lo que tampoco se considerarán en el análisis.

Por otro lado, existen 6 columnas llamadas "Clasificación" de las cuales no se tiene mayor información. Dado que podrían ser categorías importantes, se analizan sus datos para determinar a qué corresponden. 

A continuación una explicación de cada una de ellas, comenzando por la clasificación 2 para una mejor comprensión de las demás:

  - Clasificación 2: Corresponde al tipo de universidad: si es CRUCH, Privada, Instituto. FFAA, etc.
  - Clasificación 1: Es una versión menos granular de la clasificación 2, por lo que se decide eliminarla.
  - Clasificación 3: Acreditación de la carrera (Acreditada, No Acreditada)
  - Clasificación 4: Si es una organización autónoma, cerrada, de supervisión, licenciamiento, etc.
  - Clasificación 5: Si está o no adscrita a gratuidad
  - Clasificación 6: Subsistema al que pertenece la carrera

Adicionalmente, existen otras 5 columnas que también podrían ser importantes.
Estas son:
  - Horario: Existen carreras diurnas y vespertinas
  - Tipo Programa: Algunas carreras tienen un programa especial (FFAA)
  - Ingreso Directo: Algunas carreras no tienen ingreso directo (FFAA)
  - Pregrado/Posgrado: El 83% de los registros corresponden a pregrado y el 17% a posgrado
  - Tipo Moneda: Algunas carreras tienen un arancel en UF y otras en pesos


## III. Preguntas de interés
Preguntas a que se podrían resolver usando la minería de datos:
1. ¿Es posible predecir el puntaje de corte esperado para el próximo año a partir de las tendencias históricas?
2. ¿Qué factores determinan el arancel de una carrera?
3. ¿Existen universidades con características comunes? 
4. ¿Se pueden agrupar instituciones/carreras usando estas características? ¿Acaso tienen perfiles similares de aranceles y puntajes de ingreso?

Resulta interesante construir un predictor de puntajes de corte creado con datos reales, y no con simples estimaciones, o analizar el perfil de las instituciones de acuerdo a las características que se puedan extraer a partir de los atributos de la base de datos, ya que mediante simple análisis visual de las bases no siempre es directa la relación entre características de miles de datos disponibles.

Estas 4 preguntas pueden ser catalogadas en 2 grupos: preguntas de predicción y preguntas de agrupamiento (clustering).

A partir del estudio inicial del dataset se cree que hay datos suficientes para responder el primer grupo de preguntas antes planteadas, sin embargo, para poder encontrar más información sobre las características de las instituciones y carreras, se decide buscar más información en otras fuentes de datos.

## IV. Propuesta de Metodología Experimental
Considerando que nuestras preguntas serán estudiadas en dos categorías, se propone las siguientes metodologías:

**Predicción:**
- Realizar preprocesamiento: Botar columnas irrelevantes, rellenar nulos, etc.
- Fijar una carrera/clase de carrera específica: Considerando la cantidad de carreras existentes, se propone fijar una carrera específica para realizar la predicción de puntaje de corte.
- Usar modelos de regresión para predecir el puntaje de corte anual: Se debe determinar qué tipo de regresión conviene usar en función de los datos: lineal, polinomial, etc.
- Evaluar el modelo y ver qué features son importantes

**Clustering:**
- Preprocesamiento: Análogo al anterior
- Determinar número óptimo de clusters
- Usar algoritmos de clustering: K-means, DBSCAN, etc.
- Reducción de dimensiones: Se puede usar PCA para reducir la dimensionalidad de los datos y así poder visualizarlos en 2D
- Evaluación y análisis: Se debe evaluar qué tan bien se agrupan los datos y analizar los grupos obtenidos para determinar si tienen sentido o no.

## V. Experimentación

### Predicción de puntaje de corte

### Clustering de carreras e instituciones
Como se mencionó anteriormente, el conjunto de datos inicial no parece ser suficiente para responder las preguntas de agrupamiento. Por tanto, se decide utilizar una base de datos con información de los inmuebles de las instituciones de educación superior, su cantidad de laboratorios y la cantidad de docentes que trabajan en ellas.

Partiendo por el conjunto inicial y recordando el desbalance de la base de datos, se decide utilizar solo las carreras de Pregrado (83.8% de los datos). Con esto, algunas columnas del conjunto de datos "docentes" no son relevantes, como el "N°MagísterJornadaHora" (que además tiene una tasa de nulos del 33%) o "N°DoctorJornadaHora" (con una tasa de nulos del 61%) por lo que se decide eliminarlas.

Luego, como se mencionó anteriormente, se decide eliminar las columnas que no aportan información relevante para el análisis, como el orden geográfico de la región, el idgenerocarrera y el Código SIES. En cuanto a los conjuntos de datos "inmuebles", "labs" y "docentes"...

# Anexo

In [ ]:
from pathlib import Path
import joblib

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline

from pipeline import (
    DropColumns,
    DropHighNAPercentage,
    NormalizeCurrency,
    OrdinalColumnMapper,
)

pd.set_option('display.max_columns', 100)

In [ ]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"
assert DATA_DIR.exists()

DATA_PATH = DATA_DIR / "base_indices_2005-2023.xlsx"

In [ ]:
# # xlsx to pkl
df = pd.read_excel(DATA_PATH)
joblib.dump(df, DATA_DIR / 'base_indices.pkl')

In [ ]:
df = joblib.load(DATA_DIR / "base_indices.pkl")
df = df.reset_index()

## Exploración

### Hito 1

In [ ]:
df.info()

In [ ]:
# Plot distribution of pregrado vs posgrado in a pie chart
conteo_pregrado = df[df['Pregrado/Posgrado'] == 'Pregrado'].shape[0]
conteo_posgrado = df[df['Pregrado/Posgrado'] == 'Posgrado'].shape[0]

# Crear un gráfico de sectores
labels = ['Pregrado', 'Posgrado']
sizes = [conteo_pregrado, conteo_posgrado]
colors = ['lightskyblue', 'lightcoral']

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribución de Pregrado vs Posgrado')
plt.show()

In [ ]:
df['Nombre Region'].value_counts().plot(kind='barh')
plt.ylabel('Región')
plt.xlabel('Cantidad de estudiantes')
plt.title('Cantidad de estudiantes por región')

In [ ]:
df['Tipo Carrera'].value_counts().plot(kind='barh')
plt.ylabel('Tipo de carrera')
plt.xlabel('Cantidad de estudiantes')
plt.title('Cantidad de estudiantes por tipo de carrera')

In [ ]:
df['Area Conocimiento'].value_counts().plot(kind='barh')
plt.ylabel('Área de conocimiento')
plt.xlabel('Cantidad de estudiantes')
plt.title('Cantidad de estudiantes por área de conocimiento')

In [ ]:
df['Año'] = pd.to_datetime(df['Año'], format='%Y')
df.groupby("Año")['Promedio Puntaje (promedio matemáticas y lenguaje)'].max()

In [ ]:
# Datos PSU y PTU
datos_puntaje_PSU_PTU=df[df['Año'].dt.year<2023]['Promedio Puntaje (promedio matemáticas y lenguaje)'].value_counts().sort_index().plot()
plt.xlabel("Promedio puntaje")
plt.ylabel("Cantidad de estudiantes")
plt.title("Distribución de promedio de puntaje para la PSU-PTU de matemáticas y lenguaje")
plt.show()

# Datos PAES
datos_puntaje_PAES=df[(df['Año'].dt.year>=2023)]['Promedio Puntaje (promedio matemáticas y lenguaje)'].value_counts().sort_index().plot()
plt.xlabel("Promedio puntaje")
plt.ylabel("Cantidad de estudiantes")
plt.title("Distribución de promedio de puntaje para la PAES de matemáticas y lenguaje")

plt.show()

### Hito 2

In [ ]:
df['Orden Geográfico de la Región (Norte aSur)'].value_counts()

In [ ]:
# show percentage of null values in Mención o Especialidad
df['Mención o Especialidad'].isnull().sum() / df.shape[0]

In [ ]:
# show unique values in idgenerocarrera
df['idgenerocarrera'].value_counts()

In [ ]:

df['Códgo SIES'].value_counts()

In [ ]:
# show percentage of null values in each column and sort them in descending order
nulls = df.isnull().sum() / len(df) * 100
nulls.sort_values(ascending=False)


In [ ]:
interes1 = [
    "Clasificación1",
    "Clasificación2",
    "Clasificación3",
    "Clasificación4",
    "Clasificación5",
    "Clasificación6",
]
for column in interes1:
    print(f"Valores únicos de la columna {column}: \n \n{df[column].unique()} \n ")

interes2 = [
    "Horario",
    "Tipo Programa",
    "IngresoDirecto",
    "Tipo Moneda",
    "Pregrado/Posgrado",
]
for column in interes2:
    print(f"Valores únicos de la columna {column}: \n \n{df[column].unique() } \n ")
